In [1]:
import gym      # environment 라이브러리
from reinforce import *


In [2]:
from torch.distributions import Categorical

In [3]:
def main():
    # environment를 생성한다. 
    env = gym.make('CartPole-v1')
    # Policy 클래스의 인스턴스를 생성한다.
    pi = Policy()
    score = 0.0
    print_interval = 20

    # episode를 만 번 시뮬레이션 한다.
    for n_epi in range(10000):
        # env를 처음 상태로 초기화함과 동시에 observation 결과, 즉 state를 돌려준다.
        # CartPole의 경우 state는 네 개의 실수로 구성되어 있다. 
        # CartPole 환경의 자세한 사항은 https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py 에서 확인.
        s = env.reset()
        done = False

        while not done:
            # numpy.ndarray인 s를 tensor로 바꿔서 policy input으로 넣어준다. 그럼 output으로 확률분포를 얻는다. 
            # CartPole의 경우 action이 2개이다. 왼쪽으로 밀기, 오른쪽으로 밀기
            # 예를 들어 (왼쪽으로 밀 확률 0.8, 오른쪽으로 밀 확률 0.2) 와 같이 확률 분포가 주어진다.
            prob = pi(torch.from_numpy(s).float())
            # policy가 stochastic policy이므로 sampling을 해야 한다.
            # pytorch의 Categorical은 확률분포 모델이다.
            m = Categorical(prob)
            # 이 모델에서 sample을 호출하면 확률분포에 맞게 action을 tensor로 뽑아준다.
            a = m.sample()
            # env.step에 action을 주면 그 결과의 observation을 얻는다. =state transition
            # a.item()은 tensor에서 scalar를 추출하기 위해 호출한 것이다.
            s_prime, r, done, info = env.step(a.item())
            # REINFORCE 알고리즘은 return이 필요하기 때문에 에피소드가 끝나야 학습할 수 있다.
            # for 문을 돌면서 얻는 경험을 policy에 쌓아두기만 한다.
            # (현재 reward, 현재 action을 선택할 확률)
            pi.put_data((r, prob[a]))
            s = s_prime
            # score는 reward의 누적인데, reward는 매 스텝을 버틸 때마다 +1이 주어진다.
            score += r

        # 에피소드가 종료되었으므로 학습시킨다.
        pi.train_net()

        if n_epi % print_interval == 0 and n_epi != 0:
            # 20 에피소드 평균 score를 출력함.
            print(f"# of epi: {n_epi}, avg score: {score/print_interval}")
            score = 0.0
    env.close()

In [4]:
if __name__ == '__main__':
    main()

# of epi: 20, avg score: 20.5
# of epi: 40, avg score: 29.15
# of epi: 60, avg score: 27.85
# of epi: 80, avg score: 27.45
# of epi: 100, avg score: 40.15
# of epi: 120, avg score: 28.95
# of epi: 140, avg score: 38.45
# of epi: 160, avg score: 43.25
# of epi: 180, avg score: 41.7
# of epi: 200, avg score: 36.7
# of epi: 220, avg score: 46.7
# of epi: 240, avg score: 44.05
# of epi: 260, avg score: 49.75
# of epi: 280, avg score: 56.6
# of epi: 300, avg score: 63.15
# of epi: 320, avg score: 65.7
# of epi: 340, avg score: 45.4
# of epi: 360, avg score: 76.85
# of epi: 380, avg score: 76.5
# of epi: 400, avg score: 62.4
# of epi: 420, avg score: 120.7
# of epi: 440, avg score: 84.75
# of epi: 460, avg score: 123.0
# of epi: 480, avg score: 128.1
# of epi: 500, avg score: 138.95
# of epi: 520, avg score: 162.45
# of epi: 540, avg score: 166.3
# of epi: 560, avg score: 116.75
# of epi: 580, avg score: 149.6
# of epi: 600, avg score: 149.55
# of epi: 620, avg score: 163.55
# of epi: 640, a

KeyboardInterrupt: 